### Cargado de datos

Este es el codigo con el que se van a cargar los datos, estos datos se van a cargar desde un JSON

In [2]:
import json

with open('../data/datos2.json') as file:
    data = json.load(file)

data

{'nodes': [{'nombre': 'Nodo1',
   'carrera': 'Ingeniería',
   'calle': 'Calle 1',
   'semaforo': True,
   'punto_de_interes': True,
   'conexiones': ['Nodo2', 'Nodo4']},
  {'nombre': 'Nodo2',
   'carrera': 'Medicina',
   'calle': 'Calle 2',
   'semaforo': True,
   'punto_de_interes': False,
   'conexiones': ['Nodo1', 'Nodo3', 'Nodo5']},
  {'nombre': 'Nodo3',
   'carrera': 'Arquitectura',
   'calle': 'Calle 3',
   'semaforo': False,
   'punto_de_interes': True,
   'conexiones': ['Nodo2', 'Nodo6']},
  {'nombre': 'Nodo4',
   'carrera': 'Biología',
   'calle': 'Calle 4',
   'semaforo': True,
   'punto_de_interes': False,
   'conexiones': ['Nodo1', 'Nodo5', 'Nodo7']},
  {'nombre': 'Nodo5',
   'carrera': 'Economía',
   'calle': 'Calle 5',
   'semaforo': False,
   'punto_de_interes': True,
   'conexiones': ['Nodo2', 'Nodo4', 'Nodo8']},
  {'nombre': 'Nodo6',
   'carrera': 'Psicología',
   'calle': 'Calle 6',
   'semaforo': True,
   'punto_de_interes': False,
   'conexiones': ['Nodo3', 'Nodo7',

### Objeto que se tiene que crear

El objeto Nodo o punto que se tiene que crear y con el metodo para cargar datos y crear un arreglo de objetos

In [40]:
class Nodo:
    def __init__(self, nombre, carrera, calle, semaforo, punto_de_interes, conexiones):
        self.nombre = nombre
        self.carrera = carrera
        self.calle = calle
        self.semaforo = semaforo
        self.punto_de_interes = punto_de_interes
        self.conexiones = conexiones
        self.g = 0
        self.h = 0
        self.f = 0
        self.parent = None

    def __eq__(self, other):
        return self.nombre == other.nombre

    def __hash__(self):
        return hash(self.nombre)

    def load_nodes(data):
        nodes = []
        # Diccionario para mapear los nombres de nodos a objetos Nodo
        node_map = {}
        for nodo_data in data["nodes"]:
            nodo = Nodo(
                nodo_data["nombre"],
                nodo_data["carrera"],
                nodo_data["calle"],
                nodo_data["semaforo"],
                nodo_data["punto_de_interes"],
                [],
            )
            nodes.append(nodo)
            # Agregamos el nodo al mapeo
            node_map[nodo.nombre] = nodo
        # Se agregan las conexiones reales
        for nodo_conexiones in nodes:
            conexiones_nombre = [
                n_conexiones["conexiones"]
                for n_conexiones in data["nodes"]
                if n_conexiones["nombre"] == nodo_conexiones.nombre
            ]
            nodo_conexiones.conexiones = [
                node_map[nombre]
                for nombre in (conexiones_nombre[0] if conexiones_nombre else [])
            ]
        return nodes

In [42]:
# Este seria el ejemplo de uso para saber si se cargaron bien los datos

# Arreglos de nodos (Ya nuestros datos cargados y convertidos en objetos de datos)
nodes_list = Nodo.load_nodes(data=data)

# Para verificar que todo cargo bien hacemos un for para verificar dentro del array

for node in nodes_list:
    print(node.nombre)
    print(node.conexiones)
    print([n.nombre for n in node.conexiones])

Nodo1
[<__main__.Nodo object at 0x73a09a10f100>, <__main__.Nodo object at 0x73a09abd32b0>]
['Nodo2', 'Nodo4']
Nodo2
[<__main__.Nodo object at 0x73a09a10e0e0>, <__main__.Nodo object at 0x73a09abd1720>, <__main__.Nodo object at 0x73a09abd1930>]
['Nodo1', 'Nodo3', 'Nodo5']
Nodo3
[<__main__.Nodo object at 0x73a09a10f100>, <__main__.Nodo object at 0x73a09abd12a0>]
['Nodo2', 'Nodo6']
Nodo4
[<__main__.Nodo object at 0x73a09a10e0e0>, <__main__.Nodo object at 0x73a09abd1930>, <__main__.Nodo object at 0x73a09abd31f0>]
['Nodo1', 'Nodo5', 'Nodo7']
Nodo5
[<__main__.Nodo object at 0x73a09a10f100>, <__main__.Nodo object at 0x73a09abd32b0>, <__main__.Nodo object at 0x73a09abd1ab0>]
['Nodo2', 'Nodo4', 'Nodo8']
Nodo6
[<__main__.Nodo object at 0x73a09abd1720>, <__main__.Nodo object at 0x73a09abd31f0>, <__main__.Nodo object at 0x73a09abd1960>]
['Nodo3', 'Nodo7', 'Nodo9']
Nodo7
[<__main__.Nodo object at 0x73a09abd32b0>, <__main__.Nodo object at 0x73a09abd12a0>, <__main__.Nodo object at 0x73a09abd1ab0>]
['N

In [ ]:
import heapq

def calculate_heuristic(node, end):
    return abs(node.conexiones.index(end) - node.conexiones.index.end(end))


def astar(start, end):
    open_list = []
    closed_list = set()

    heapq.heappush(open_list, (0, start))

    while open_list:
        current_node = heapq.heappop(open_list)[1]
        closed_list.add(current_node)
        print(end)
        if current_node == end:
            path = []
            while current_node:
                path.append(current_node.nombre)
                current_node = current_node.parent
            return path[::-1]

        for neighbor in current_node.conexiones:
            if neighbor in closed_list:
                continue
            print('Valor g', type(neighbor.g))
            neighbor.g = current_node.g + 1
            neighbor.h = calculate_heuristic(neighbor, end)
            neighbor.f = neighbor.g + neighbor.h

            for _, open_node in open_list:
                if neighbor == open_node and neighbor.g > open_node.g:
                    continue

            neighbor.parent = current_node
            heapq.heappush(open_list, (neighbor.f, neighbor))

    return None

In [ ]:
path = astar(nodes_list[0], nodes_list[-1])
print(path)

Valor g <class 'int'>


TypeError: unsupported operand type(s) for -: 'str' and 'str'